In [1]:
from head import blip_vqa

In [2]:
model = blip_vqa.BLIP_VQA(vit='base_8')

In [1]:
from backbone.utils import _download, _MODELS
_download(_MODELS['base_8'] ,'/Users/valery/GitHub/VideoQA-NTO/models')
_download('https://huggingface.co/bert-base-multilingual-uncased/resolve/main/pytorch_model.bin' ,'/Users/valery/GitHub/VideoQA-NTO/models')

In [3]:
import torch
checkpoint = torch.load('/Users/valery/GitHub/VideoQA-NTO/models/k600_k710_uniformerv2_b16_8x224.pyth', map_location='cpu') 

ww = {}
for key in checkpoint.keys():
    ww[key.replace('backbone', 'visual_encoder')] = checkpoint[key]

msg = model.load_state_dict(ww,strict=False)

In [4]:
import torch
checkpoint = torch.load('/Users/valery/GitHub/VideoQA-NTO/models/pytorch_model.bin', map_location='cpu') 

ww_encoder = {}
for key in model.state_dict().keys():
    key = key.replace('text_encoder', 'bert')
    if key in checkpoint.keys():
        if 'word_embeddings' in key:
            del checkpoint[key]
        else:
            ww_encoder[key.replace('bert', 'text_encoder')] = checkpoint[key]
# for key in checkpoint.keys():
    # if 'word_embeddings' in key:
    #     # checkpoint[key.replace('bert', 'text_encoder')] = model.state_dict()[key.replace('bert', 'text_encoder')]
    # ww_encoder[key.replace('bert', 'text_encoder')] = checkpoint[key]

msg1 = model.load_state_dict(ww_encoder,strict=False)

ww_decoder = {}
for key in checkpoint.keys():
    ww_decoder[key.replace('bert', 'text_decoder.bert')] = checkpoint[key]

msg2 = model.load_state_dict(ww_decoder,strict=False)

In [5]:
key

'cls.seq_relationship.bias'

In [1]:
import torch
video = torch.randn(1, 3, 8, 224, 224)
question = 'who was gay?'
answer = 'adam'
n = 1
weights = 0.2



In [7]:
outputs = model(video, question, answer, [n] , weights, train=False)

In [8]:
outputs

['нацыянальны jobs ש jonathanasne rekyhanyasne pistas']

In [25]:
import pandas as pd

df = pd.read_csv('/Users/valery/GitHub/VideoQA-NTO/activitynet/updatedtrain.csv')
df = df[(df['answer'] != 'да') & (df['answer'] != 'нет')]
df['video_name'] = 'activitynet/videos/' + df['video_name']

In [26]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [27]:
train.to_csv('/Users/valery/GitHub/VideoQA-NTO/activitynet/train.csv', index=False)
test.to_csv('/Users/valery/GitHub/VideoQA-NTO/activitynet/test.csv', index=False)

In [28]:
import pandas as pd
df = pd.read_csv('/Users/valery/GitHub/VideoQA-NTO/activitynet/test.csv')

In [29]:
df

,Unnamed: 0,video_name,question,answer
0,594,activitynet/videos/wNvLv7PtvQg,как сложно кататься на лодке,обычно
1,12381,activitynet/videos/3UgGvKnelfY,почему человек носит шлем,защитная головка
2,9315,activitynet/videos/QHJTOHgjDMw,что делает человек в сером жилете,играть на пианино
3,9342,activitynet/videos/kdRcWkrZRZo,что делает человек в белой рубашке,выпуск новостей
4,15280,activitynet/videos/IWHER8iS1B4,что спереди у детей в радужной одежде,аудитория
...,...,...,...,...
3794,14122,activitynet/videos/OIA7lPraPSM,почему мужчина на видео меняет позу,готов к прыжку
3795,15375,activitynet/videos/_4CLYKFzmoY,что на ухе у человека в гарнитуре,гарнитура
3796,14298,activitynet/videos/s82_J03bqwQ,почему человек в красных рубашках полирует перед,гладкая поверхность
3797,4839,activitynet/videos/qRT0Z6Sz_Ck,что делает человек в клетчатой ​​рубашке,снять видео


In [23]:
from iopath.common.file_io import g_pathmgr
import os

_num_clips = 2
mode = 'test'
path_to_file = os.path.join(
    '/Users/valery/GitHub/VideoQA-NTO/activitynet', "{}.csv".format(mode)
)

df = pd.read_csv(path_to_file)

_path_to_videos = df['video_name'].to_list()
_labels = df['answer'].to_list()
_question = df['question'].to_list()

for idx in range(_num_clips - 1):
    _path_to_videos = _path_to_videos *2
    _labels = _labels *2
    _question = _question *2

answer_weight = {}
for answer in _labels:
    if answer in answer_weight.keys():
        answer_weight[answer] += 1/len(_labels)
    else:
        answer_weight[answer] = 1/len(_labels)

    weights = list(answer_weight.values())


